# Hyperparameter Tuning using HyperDrive

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'cifar-hyperdrive-experiment'
exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Attach an AmlCompute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Use GPU instance because it is better suited for computer vision problems
amlcompute_instance = "cifar-instance"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_instance)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                           vm_priority='lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_instance, compute_config)

compute_target.wait_for_completion(show_output=True)

## Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Reference: https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
from azureml.core import Dataset

# URL to the CIFAR-10 dataset
dataset_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

# Register the dataset using the Dataset class
dataset_name = "cifar10_dataset"
cifar10_dataset = Dataset.File.from_files(dataset_url)
cifar10_dataset.register(workspace=ws, name=dataset_name, create_new_version=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.core import ScriptRunConfig, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, normal, uniform
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy

sampling = RandomParameterSampling({
    "--dropout1": uniform(0.0, 0.1),
    "--dropout2": uniform(0.2, 0.5),
    "--dropout3": normal(0.3, 0.7), # Prefering normal distribution to avoid extremes
    "--epochs": choice(10, 20, 40)
})

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)
environment = Environment.get(ws, name='aml-environment')
script = ScriptRunConfig(source_directory='.',
                         script='cifar10_train.py',
                         compute_target=amlcompute_instance,
                         environment=environment)

hyperdrive_config = HyperDriveConfig(run_config=script,
                    hyperparameter_sampling=sampling,
                    policy=policy,
                    primary_metric_name='accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=10,
                    max_concurrent_runs=4)

In [ ]:
# Submit the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(f"Best run: {best_run}\n")
print(f"Best run metrics: {best_run_metrics}\n")
print(f"Associated files: {best_run.get_file_names()}\n")

In [ ]:
# TODO: Download the model
# Downloading model from the run into the workspace
best_run.download_file('models/cifar10-cnn-hyperdrive.keras')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model

# TODO: Change the `model_path`
model = Model.register(model_path="*.keras",
                       model_name="cirfar10-best-model",
                       tags={'topic': "computer_vision", 'type': "classification"},
                       description="CNN model on CIFAR-10 dataset to classify 32x32 images.",
                       workspace=ws)

TODO: In the cell below, send a request to the web service you deployed to test it.

### Deploy an Azure Container Instance Web Service

In [ ]:
# Uncomment the following code if you want customize the deployment configuration
# from azureml.core.webservice import AciWebservice
# aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_name = 'cifar10-service'

# Use the `deployment_config` argument for custom config
service = Model.deploy(ws, service_name, [model], overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

### Test the Web Service Endpoint

In [ ]:
from utils import visualize
# TODO: Replace it with the consume script provided by AML studio
# Get input data (use the dumped json from the inference script) -> Show as img -> Inference
input_data = {
    "input_key": input_value
}

# Call the web service endpoint
result = service.run(input_data)
print("Web service response:", result)

# TODO: Get the predicted labels ID
predicted_labels = ...
# TODO: Get the test labels ID
test_labels = ...

visualize[result[0]]
print("Predicted labels: ", predicted_labels[0])
print("Ground truth: ", test_labels[0])

## Delete the Resource
TODO: In the cell below, print the logs of the web service and delete the service
- webservice
- cluster

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

